# Option Pricing and Stochastic Volatility using Neural Network

## Simulated Training Data Collection

In [23]:
m, n, T = 1000, 252, 1

def eulers_discretion_simulation(s0: float, v0: float, kappa: float, theta: float, sigma: float, t: float, m: int, n: int, r: float) -> tuple:
    """
    Inputs:
     - s0, v0: Initial parameters for asset and variance.
     - kappa: Rate of mean reversion in variance process.
     - theta: Long-term mean of variance process.
     - sigma: Vol of vol / volatility of variance process.
     - T: Time of simulation.
     - N: Number of time steps.
     - M: Number of scenarios / simulations.
     - R: Risk Free Rate
    Outputs:
     - S: Asset prices over time (array).
     - V: Variance over time (array).
    """

    dt = t / m
    S = np.zeros((m, n + 1))
    V = np.zeros((m, n + 1))
    np.random.seed(1926159)
    for i in range(m):
        S[i, 0] = s0
        V[i, 0] = v0
        for j in range(1, n + 1):
            Z1 = np.random.randn()
            Z2 = np.random.randn()

            dS = r * S[i, j - 1] * dt + np.sqrt(V[i, j - 1] * dt) * Z1
            dV = kappa * (theta - V[i, j - 1]) * dt + sigma * np.sqrt(V[i, j - 1] * dt) * Z2

            S[i, j] = S[i, j - 1] + dS
            V[i, j] = V[i, j - 1] + dV

    return S, V

S, V = eulers_discretion_simulation(100, 0.1, 2.0, 0.1, 0.2, 1, 1000, 252, 0.05)

## Data Pre-Processing

In [24]:
import numpy as np

def format_data(data: list, sequence_length: int) -> tuple:
    """
    Inputs:
    - data: Simulated Heston Data.
    - sequence_length: Considered Time Steps.
    Return:
    - Testing and Training Data.
    """
    x, y = [], []
    for i in range(len(data) - sequence_length):
        x.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(x), np.array(y)

In [25]:
sequence_length = 10

flat_S = S.flatten()
flat_S = (flat_S - np.mean(flat_S)) / np.std(flat_S)

split_ratio = 0.8
split_index = int(len(flat_S) * split_ratio)

train_data = flat_S[:split_index]
test_data = flat_S[split_index:]

x_train, y_train = format_data(train_data, sequence_length)
x_test, y_test = format_data(test_data, sequence_length)

## Neural Network Implementation

In [26]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(
    hidden_layer_sizes=(100,100,100,100),
    solver='adam',
    shuffle = False,
    batch_size=64,
    verbose=True,
    max_iter= 25
)

## Hyperparameter Tuning

In [27]:
from sklearn.model_selection import GridSearchCV

model.get_params()

parameter_space = {
    'activation': ['tanh', 'relu'],
    'alpha': [0.01, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf = GridSearchCV(model, parameter_space, n_jobs=-1, cv=3)
clf.fit(x_train, y_train)

print('Best parameters found:\n', clf.best_params_)

/Users/harvey/Documents/Portfolio/Option Pricing and Stochastic Volatility/environment/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/harvey/Documents/Portfolio/Option Pricing and Stochastic Volatility/environment/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/harvey/Documents/Portfolio/Option Pricing and Stochastic Volatility/environment/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.03436561
Iteration 2, loss = 0.02577214
Iteration 3, loss = 0.02409512
Iteration 4, loss = 0.02358996
Iteration 5, loss = 0.02334735
Iteration 6, loss = 0.02319992
Iteration 7, loss = 0.02311617
Iteration 8, loss = 0.02304726
Iteration 9, loss = 0.02300418
Iteration 10, loss = 0.02296550
Iteration 11, loss = 0.02293039
Iteration 12, loss = 0.02290741
Iteration 13, loss = 0.02289680
Iteration 14, loss = 0.02287996
Iteration 15, loss = 0.02286594
Iteration 16, loss = 0.02285808
Iteration 17, loss = 0.02286069
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


GridSearchCV(cv=3,
             estimator=MLPRegressor(batch_size=64,
                                    hidden_layer_sizes=(100, 100, 100, 100),
                                    max_iter=25, shuffle=False, verbose=True),
             n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'], 'alpha': [0.01, 0.05],
                         'learning_rate': ['constant', 'adaptive']})

## Network Training

In [28]:
import random
random.seed(1926159)

model.fit(x_train, y_train)

Iteration 1, loss = 0.02485644
Iteration 2, loss = 0.02307125
Iteration 3, loss = 0.02279475
Iteration 4, loss = 0.02264507
Iteration 5, loss = 0.02254826
Iteration 6, loss = 0.02245632
Iteration 7, loss = 0.02241325
Iteration 8, loss = 0.02235508
Iteration 9, loss = 0.02230003
Iteration 10, loss = 0.02225789
Iteration 11, loss = 0.02222781
Iteration 12, loss = 0.02221202
Iteration 13, loss = 0.02219889
Iteration 14, loss = 0.02217823
Iteration 15, loss = 0.02216350
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPRegressor(batch_size=64, hidden_layer_sizes=(100, 100, 100, 100),
             max_iter=25, shuffle=False, verbose=True)

In [29]:
import pickle

filename = 'neural_network_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [30]:
filename = 'neural_network_model.sav'
model = pickle.load(open(filename, 'rb'))

## Monte Carlo Simulation

In [32]:
from heston_monte_carlo import heston_model_monte_carlo_simulation

S, V = heston_model_monte_carlo_simulation(100, 0.1, 0, 2.0, 0.1, 0.2, 1, 1000, 252)

sequence_length = 10

flat_S = S.flatten()
flat_S = (flat_S - np.mean(flat_S)) / np.std(flat_S)

split_ratio = 0.8
split_index = int(len(flat_S) * split_ratio)

train_data = flat_S[:split_index]
test_data = flat_S[split_index:]

x_predicted, monte_carlo_y_predicted = format_data(test_data, sequence_length)

ValueError: Found input variables with inconsistent numbers of samples: [50590, 50441]

## Testing And Analysis

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
import pandas as pd

neural_network_y_predicted = model.predict(x_test)

def performance_analysis(true: list, predicted: list):
    """
    Inputs:
    - true: The real value.
    - predicted: The model predicted value.
    Return:
    - Results Data Frame.
    """
    mse = mean_squared_error(true, predicted)
    rmse = sqrt(mse)
    mae = mean_absolute_error(true, predicted)
    r2 = r2_score(true, predicted)
    df = {'Mean Squared Error': [mse], 'Root Mean Squared Error': [rmse], 'Mean Absolute Error': [mae], 'R2 Score': [r2]}
    return pd.DataFrame(data=df)

In [ ]:
print("Monte Carlo Results:")
print(performance_analysis(y_test, monte_carlo_y_predicted))
print("Neural Network Results:")
print(performance_analysis(y_test, neural_network_y_predicted))